In [238]:
import pandas as pd

with open('API_Request_Output.json') as json_data:
    raw_data = json.load(json_data)
    
    # Extracting only the time series
    df = pd.DataFrame(raw_data['series'])
    
    # Drop columns which are not of interest
    df = df.drop(['customerId', 'status', 'assetId', 'currency', 'endInterval', 'resolution'], axis=1)
    
    # Extracting positions and converting them to floats
    df['positions'] = df['positions'].apply(lambda x: [d['quantity'] for d in x if 'quantity' in d])

    # Convert 'startInterval' from string to pandas time module
    df['startInterval'] = pd.to_datetime(df['startInterval'])

# Function to interate the dataframe over each quantity, making a new row for each quantity and hour
def dataframe_iteration(row):
    return [[row['externalId'], pos, row['price'], row['priceArea'], row['direction'], row['startInterval']] for pos in row['positions']]

# Executing the function
new_df = df.apply(lambda row: dataframe_iteration(row), axis=1)

# Expand lists into separate rows
new_df = new_df.explode().reset_index(drop=True)

# Creates a sequence number for each row within the same Id
new_df = pd.DataFrame(new_df.tolist(), columns=['Id', 'Positions', 'Price', 'Area', 'Direction', 'startInterval'])

# Adds a new column 'Time' which increases by 1 hour for each iteration of 'Positions'
new_df['Time'] = new_df.groupby('Id').cumcount().apply(lambda x: pd.Timedelta(hours=x)) + new_df['startInterval']

# Drop 'startInterval' column
new_df = new_df.drop(columns=['startInterval'])

# Convert '2024-02-03 02:00:00' format to 'Date' and 'Time' for easier data processing
new_df['Time'] = pd.to_datetime(new_df['Time'])
new_df['Date'] = new_df['Time'].dt.date
new_df['Time'] = new_df['Time'].dt.strftime('%H')

# Convert quantities in 'Positions' to negative if the 'Direction' is Down
new_df.loc[new_df['Direction'] == 'Down', 'Positions'] *= -1

# Drop 'Direction' column
new_df = new_df.drop(columns=['Direction'])

new_df.to_csv('Processed_Output.csv', index=False)